In [1]:
import pandas as pd
import numpy as np

In [2]:
#use csv with already preprocessed features (scaling, differnecing etc needed)
df = pd.read_csv("Data_Prep.csv", index_col=0)

In [3]:
df.reset_index() [["index", "Total_Heat"]]

,index,Total_Heat
0,2008-01-01 00:00:00+00:00,98086.0
1,2008-01-01 01:00:00+00:00,100454.0
2,2008-01-01 02:00:00+00:00,107183.0
3,2008-01-01 03:00:00+00:00,120688.0
4,2008-01-01 04:00:00+00:00,158569.0
...,...,...
61363,2014-12-31 19:00:00+00:00,153908.0
61364,2014-12-31 20:00:00+00:00,139544.0
61365,2014-12-31 21:00:00+00:00,114048.0
61366,2014-12-31 22:00:00+00:00,71690.5


In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
# Preprocessing for deepAR
my_df = df.reset_index() [["index", "Total_Heat"]]
my_df.columns = ["date", "y"]
my_df.ds = pd.to_datetime(my_df.ds).dt.tz_convert(None)

AttributeError: 'DataFrame' object has no attribute 'ds'

In [ ]:
df_scaler = MinMaxScaler()
my_df.y = pd.DataFrame(df_scaler.fit_transform(my_df[["y"]]), columns=["y"])


In [7]:
from gluonts.dataset.common import ListDataset
start = pd.Timestamp("01-01-2008", freq="H")
# train dataset: cut the last window of length "prediction_length", add "target" and "start" fields
train_ds = ListDataset([{'target': my_df.loc[:450,'y'], 'start': start}], freq='H')
# test dataset: use the whole dataset, add "target" and "start" fields
test_ds = ListDataset([{'target': my_df['y'], 'start': start}],freq='H')

C:\Users\jabbe\AppData\Local\Temp\ipykernel_27220\2368371765.py:2: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  start = pd.Timestamp("01-01-2008", freq="H")


In [1]:
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
# import mxnet as mx
import numpy as np

np.random.seed(7)

estimator = DeepAREstimator(
    prediction_length=28,
    context_length=100,
    freq="H",
    trainer=Trainer(#ctx="gpu", # remove if running on windows
                    epochs=5,
                    learning_rate=1e-3,
                    num_batches_per_epoch=100
                   )
)

predictor = estimator.train(train_ds)

c:\Users\jabbe\Envs\idw\lib\site-packages\gluonts\json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


ModuleNotFoundError: No module named 'mxnet'

In [ ]:
predictions = predictor.predict(test_ds)
predictions = list(predictions)[0]
predictions = predictions.quantile(0.5)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score( list(test_ds)[0]['target'][-28:], predictions))

import matplotlib.pyplot as plt
plt.plot(predictions)
plt.plot(list(test_ds)[0]['target'][-28:])
plt.legend(['predictions', 'actuals'])
plt.show()



In [ ]:
# Hyperparameter
train_days = 30
forecast_days = 2

In [ ]:
# creating two seperate lists with a function

def create_X_y(data_df: pd.DataFrame, train_days:int, forecast_days:int):
    """"""
    X_list = []
    y_list = []

    for i in range(train_days*24, len(data_df)-forecast_days*24, 24):
        X_list.append(data_df.iloc[i-train_days*24:i])
        y_list.append(data_df[i:i+forecast_days*24])
    
    return X_list, y_list
    

In [ ]:
X_list, y_list = create_X_y(my_df[["ds", "y"]], train_days, forecast_days)